In [10]:
import pandas as pd
import requests
import time
import re
from geopy.geocoders import Nominatim

In [ ]:
df_aed = pd.read_parquet('aed_locations.parquet.gzip')

In [ ]:
df_aed['lat'] = None
df_aed['lon'] = None

## Cleaning the data

In [17]:
#cleaning abbreviations
for i in range(len(df_aed)):
    text = df_aed.loc[i, 'address']
    if type(text) == float:
        text = str(text)
    if (not text == None):
        if (re.search('Ch.',text)):
            new_text = text.replace('Ch.', 'Chaussée')
            df_aed.loc[i, 'address'] = new_text
        if (re.search('Blvd.',text)):
            new_text = text.replace('Blvd.', 'Boulevard')
            df_aed.loc[i, 'address'] = new_text
        if (re.search('Bld',text)):
            new_text = text.replace('Bld', 'Boulevard')
            df_aed.loc[i, 'address'] = new_text
        if (re.search('sestweg',text)):
            new_text = text.replace('sestweg', 'sesteenweg')
            df_aed.loc[i, 'address'] = new_text 


using Nominatim geolocator for most addresses because there is no limit for api calls as long as you respect the limit per minute

In [ ]:
geolocator = Nominatim(user_agent='Isabel')

In [ ]:
counter = 0
not_found = 0
for number in range(len(df_aed)):
    #making structure query for addresses with numbers and unstructured query for addresses without numbers
    structured_query = {}
    if not pd.isna(df_aed['postal_code'][number]):
        structured_query['postalcode']= int(df_aed.loc[number,'postal_code'])         
    structured_query['country']= 'Belgium'
    if not pd.isna(df_aed['address'][number]):
        if not pd.isna(df_aed['number'][number]):
            structured_query['street'] = f"{int(df_aed.loc[number,'number'])} {df_aed.loc[number,'address']}"
            location=geolocator.geocode(structured_query, timeout=15)
        else:
            #making unstructured search string
            address = ''
            if not pd.isna(df_aed['address'][number]):
                address = str(df_aed['address'][number])
                if not pd.isna(df_aed['number'][number]):
                    address = address+" "+str(int(df_aed['number'][number]))
                if not pd.isna(df_aed['postal_code'][number]):
                    address = address +" ,"+str(int(df_aed['postal_code'][number]))
                if not pd.isna(df_aed['municipality'][number]):
                    address = address +" ,"+str(df_aed['municipality'][number])
                address = address + ' , Belgium'
            location=geolocator.geocode(address, timeout=15)
    if location == None:
        not_found += 1
        if (not_found % 10 == 0):
            print('number of not found addresses:'+ str(not_found))
    else:
        df_aed.loc[counter,'lon'] =location.longitude
        df_aed.loc[counter,'lat'] =location.latitude
    counter += 1
    if (counter % 50 ==0):
        print('sleeping....')
        time.sleep(5)
        print('awake and count is: '+str(counter))


there are about 2000 addresses that Nominatim cannot find, so we save the obtained geodata and will then procede with another geocoder that has a stricter limit

In [ ]:
file_path = 'aed_coordinates2.csv'
df_aed.to_csv(file_path, index=False)
print("CSV file saved successfully.")

In [3]:
aed_coordinates = pd.read_csv('aed_coordinates2.csv')

In [52]:
aed_coordinates.head()

,id,type,address,number,postal_code,municipality,province,location,public,available,hours,lat,lon
0,13.0,NaN,Boulevard Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,NaN,Y,NaN,NaN,NaN,NaN
1,70.0,NaN,Chaussée De Wégimont,76.0,4630.0,Ayeneux,Liège,NaN,NaN,NaN,NaN,50.612439,5.737995
2,71.0,NaN,Place Saint - Lambert,NaN,4020.0,Liège,Liège,NaN,NaN,NaN,NaN,50.644956,5.574513
3,72.0,NaN,Rue du Doyard,NaN,4990.0,Lierneux,Liège,NaN,NaN,NaN,NaN,50.285641,5.780870
4,73.0,NaN,Fond Saint Servais,NaN,4000.0,Liège,Liège,NaN,NaN,NaN,NaN,50.646765,5.570987


In [59]:
#make addresses
aed_coordinates['full_address'] = ''
for number in range(len(aed_coordinates)):
    address = ''
    if not pd.isna(aed_coordinates['address'][number]):
        address = str(aed_coordinates['address'][number])
        if not pd.isna(aed_coordinates['number'][number]):
            address = address+" "+str(int(aed_coordinates['number'][number]))
        if not pd.isna(aed_coordinates['postal_code'][number]):
            address = address +" ,"+str(int(aed_coordinates['postal_code'][number]))
        if not pd.isna(aed_coordinates['municipality'][number]):
            address = address +" ,"+str(aed_coordinates['municipality'][number])
    aed_coordinates.loc[number,'full_address'] = address
    

## use openrouteservice geocoder to retrieve the remaining geolocations

understanding the structure

In [44]:
# Define the API endpoint and your API key
api_key = '5b3ce3597851110001cf6248d11d604663d24d9395ef7a513daed710'
url = 'https://api.openrouteservice.org/geocode/search?api_key=5b3ce3597851110001cf6248d11d604663d24d9395ef7a513daed710&text=Lindensestraat%2015,%20Leuven&boundary.country=BE'
# Define the query parameters
params = {
    'api_key': api_key
}

# Make the GET request
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")


{'geocoding': {'version': '0.2', 'attribution': 'https://openrouteservice.org/terms-of-service/#attribution-geocode', 'query': {'text': 'Lindensestraat 15, Leuven', 'size': 10, 'private': False, 'boundary.country': ['BEL'], 'lang': {'name': 'English', 'iso6391': 'en', 'iso6393': 'eng', 'via': 'default', 'defaulted': True}, 'querySize': 20, 'parser': 'libpostal', 'parsed_text': {'street': 'lindensestraat', 'housenumber': '15', 'city': 'leuven'}}, 'engine': {'name': 'Pelias', 'author': 'Mapzen', 'version': '1.0'}, 'timestamp': 1716383967618}, 'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [4.724449, 50.884538]}, 'properties': {'id': 'polyline:4777983', 'gid': 'openstreetmap:street:polyline:4777983', 'layer': 'street', 'source': 'openstreetmap', 'source_id': 'polyline:4777983', 'name': 'Lindensestraat', 'street': 'Lindensestraat', 'confidence': 0.8, 'match_type': 'fallback', 'accuracy': 'centroid', 'country': 'Belgium', 'country_

In [50]:
coordinates = data['features'][0]['geometry']['coordinates']
print(coordinates)

[4.724449, 50.884538]


get the missing longitude and latitude data

In [19]:
request_limit = 2700
request_count = 1

In [20]:
for row in range(len(aed_coordinates)):
    if request_limit == request_count:
        break
    
    #make sure to meet limit requests of 100 per minute
    if (request_count % 100 ==0):
        print(f'sleeping...., request count is {request_count}')
        time.sleep(10)
        print('awake')
        request_count += 1
    #make URL for missing coordinates
    if pd.isna(aed_coordinates.loc[row, 'lat']):
        url_start = 'https://api.openrouteservice.org/geocode/search?api_key=5b3ce3597851110001cf6248d11d604663d24d9395ef7a513daed710&text='
        url_end = '&boundary.country=BE'
        address = aed_coordinates.loc[row, 'full_address']
        new_address = address.replace(' ', '%')
        url = url_start+new_address+url_end

        #make request and increase request counter
        request_count += 1
        response = requests.get(url)
        print(f'{request_count}{response}')
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            #get the coordinates
            if not data['features'] == []:
                coordinates = data['features'][0]['geometry']['coordinates']

                #save longitude and latitude in the dataframe
                aed_coordinates.loc[row,'lon'] =coordinates[0]
                aed_coordinates.loc[row,'lat'] =coordinates[1]
        else:
            print(f"Error: {response.status_code}")

2<Response [200]>
3<Response [200]>
4<Response [200]>
5<Response [200]>
6<Response [200]>
7<Response [200]>
8<Response [200]>
9<Response [200]>
10<Response [200]>
11<Response [200]>
12<Response [200]>
13<Response [200]>
14<Response [200]>
15<Response [200]>
16<Response [200]>
17<Response [200]>
18<Response [200]>
19<Response [200]>
20<Response [200]>
21<Response [200]>
22<Response [200]>
23<Response [200]>
24<Response [200]>
25<Response [200]>
26<Response [200]>
27<Response [200]>
28<Response [200]>
29<Response [200]>
30<Response [200]>
31<Response [200]>
32<Response [200]>
33<Response [200]>
34<Response [200]>
35<Response [200]>
36<Response [200]>
37<Response [200]>
38<Response [200]>
39<Response [200]>
40<Response [200]>
41<Response [200]>
42<Response [200]>
43<Response [200]>
44<Response [200]>
45<Response [200]>
46<Response [200]>
47<Response [200]>
48<Response [200]>
49<Response [200]>
50<Response [200]>
51<Response [200]>
52<Response [200]>
53<Response [200]>
54<Response [200]>
5

KeyboardInterrupt: 

In [8]:
aed_coordinates.loc[aed_coordinates['lon'].isnull()]

,id,type,address,number,postal_code,municipality,province,location,public,available,hours,lat,lon,full_address
16,98.0,NaN,Wemmelsteenweg,100.0,1090.0,Brussel,Bruxelles-Brussel,NaN,NaN,NaN,NaN,NaN,NaN,"Wemmelsteenweg 100 ,1090 ,Brussel"
30,218.0,NaN,Cours des mineurs,NaN,4000.0,Liège,Liège,NaN,NaN,NaN,NaN,NaN,NaN,"Cours des mineurs ,4000 ,Liège"
47,235.0,NaN,Route du Canada,157.0,4910.0,La Reid,Liège,NaN,NaN,NaN,NaN,NaN,NaN,"Route du Canada 157 ,4910 ,La Reid"
73,349.0,NaN,Haven 1968 Sint Annalaan,1.0,9130.0,Kallo,Oost-Vlaanderen,NaN,Y,NaN,NaN,NaN,NaN,"Haven 1968 Sint Annalaan 1 ,9130 ,Kallo"
74,354.0,NaN,Bld Leopold II,44.0,1080.0,Bruxelles,Bruxelles-Brussel,NaN,NaN,NaN,NaN,NaN,NaN,"Bld Leopold II 44 ,1080 ,Bruxelles"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15185,15100.0,Appareil fixe-Vast apparaat,Avenue Jean Van Horebeeck,196.0,1160.0,Brussel,Bruxelles-Brussel,,Oui-Ja,Non-Nee,NaN,NaN,NaN,"Avenue Jean Van Horebeeck 196 ,1160 ,Brussel"
15190,16627.0,Appareil fixe-Vast apparaat,Avenue de Marie,175.0,7500.0,Tournai,Hainaut,,Oui-Ja,Non-Nee,NaN,NaN,NaN,"Avenue de Marie 175 ,7500 ,Tournai"
15193,16630.0,Appareil fixe-Vast apparaat,xngng,218.0,6001.0,marcinelle,Hainaut,Loge de garde,Oui-Ja,Non-Nee,06h00 à 22h00,NaN,NaN,"xngng 218 ,6001 ,marcinelle"
15214,14575.0,Appareil fixe-Vast apparaat,Carrefour de la Spetz,2.0,6700.0,Arlon,Luxembourg,Extérieur Hall 2,Non-Nee,Oui-Ja,NaN,NaN,NaN,"Carrefour de la Spetz 2 ,6700 ,Arlon"


In [85]:
# Write the dataframe to a CSV file
filepath = 'all_aed_coordinates.csv'
aed_coordinates.to_csv(filepath, index=False)
print("CSV file saved successfully.")

CSV file saved successfully.
